## 텍스트 전처리

In [1]:
# 라이브러리 호출
import os
import numpy as np
import pandas as pd
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# 현재 작업 경로 확인
os.getcwd()

'/Users/seonghona/Documents/Lectures/Customer/DBR/code'

In [3]:
# data 폴더로 작업 경로 변경
os.chdir(path = '../data')

In [4]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
sorted(os.listdir())

['Naver_News_List.pkl', 'Naver_News_Reply.pkl']

In [5]:
# pkl 파일을 읽고 newsReply 생성
newsReply = pd.read_pickle(filepath_or_buffer = 'Naver_News_Reply.pkl')

In [6]:
# newsReply의 처음 5행 확인
newsReply.head()

,objectId,commentNo,parentCommentNo,replyAllCount,contents,userName,modTime,regTime,sympathyCount,antipathyCount,hiddenByCleanbot,deleted
0,"news003,0012717267",833652862377525492,833652862377525492,5,고작 본인 책 홍보하고 사진 몇장 찍자고 파리까지 간거냐??? 축협회장이 아니라 정...,sdw8****,2024-08-08T11:35:30+0900,2024-08-08T11:35:30+0900,381,2,False,False
1,"news003,0012717267",833653336652644737,833653336652644737,3,FIFA 회장 만난다고 우리가 정몽규 지를 무슨 우러러 보고\n월드컵 못나갈까봐 전...,twin****,2024-08-08T11:42:51+0900,2024-08-08T11:42:51+0900,162,1,False,False
2,"news003,0012717267",833654375934067221,833654375934067221,1,클린스만 위약금 본인이 꼭 충당해라,boi2****,2024-08-08T11:58:59+0900,2024-08-08T11:58:59+0900,103,1,False,False
3,"news003,0012717267",833653269342454167,833653269342454167,3,몽규 비행기 호텔값은 자비로 간거니? 아님?,jjan****,2024-08-08T11:41:49+0900,2024-08-08T11:41:49+0900,54,1,False,False
4,"news003,0012717267",833653347641720850,833653347641720850,1,헤헤~~굽신굽신. 피파에 자리 좀 없슈?,lljh****,2024-08-08T11:43:02+0900,2024-08-08T11:43:02+0900,26,1,False,False


In [7]:
# newsReply의 정보 확인
newsReply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   objectId          4032 non-null   object
 1   commentNo         4032 non-null   object
 2   parentCommentNo   4032 non-null   object
 3   replyAllCount     4032 non-null   int64 
 4   contents          4032 non-null   object
 5   userName          4032 non-null   object
 6   modTime           4032 non-null   object
 7   regTime           4032 non-null   object
 8   sympathyCount     4032 non-null   int64 
 9   antipathyCount    4032 non-null   int64 
 10  hiddenByCleanbot  4032 non-null   bool  
 11  deleted           4032 non-null   bool  
dtypes: bool(2), int64(3), object(7)
memory usage: 323.0+ KB


In [8]:
# newsReply의 contents를 sens에 할당
sens = newsReply['contents']

In [9]:
# 인덱스 설정: 첫 번째 원소
# [참고] 반복문으로 실행할 코드에 인덱스를 추가하면 편리함
i = 0

In [10]:
# 원본 문서의 첫 번째 원소 확인
sens.iloc[i]

'고작 본인 책 홍보하고 사진 몇장 찍자고 파리까지 간거냐??? 축협회장이 아니라 정치인같다.ㅉㅉㅉ'

In [11]:
# sens에서 지정한 패턴이 아닌 글자를 공백으로 변경하고 sens에 재할당
sens = sens.str.replace(pat = '[^가-힣A-Za-z0-9.]|&.+;', repl = '', regex = True)

# 일부 단어를 변경함
sens = sens.str.replace(pat = '축협', repl = '축구협회', regex = True)

In [12]:
# sens의 첫 번째 원소와 비교
sens[i]

'고작본인책홍보하고사진몇장찍자고파리까지간거냐축구협회회장이아니라정치인같다.'

### 형태소 분석

In [13]:
# 내장 불용어 객체 생성
# [참고] 내장 불용어 객체의 stopwords 속성으로 불용어 목록 확인 가능
kiwistop = Stopwords()

In [14]:
# 한글 형태소 분석기 설정
# [참고] typos 매개변수에 'basic'을 지정하면 내장된 오타 정보로 문장 교정
kiwi = Kiwi()

In [15]:
# sens의 첫 번째 원소 확인
sens[i]

'고작본인책홍보하고사진몇장찍자고파리까지간거냐축구협회회장이아니라정치인같다.'

In [16]:
# sens의 첫 번째 원소로 형태소 분석
# [참고] 형태소 분석 결과에서 불용어 제거
kiwi.tokenize(text = sens[i], stopwords = kiwistop)

[Token(form='고작', tag='MAG', start=0, len=2),
 Token(form='인책', tag='NNG', start=3, len=2),
 Token(form='홍보', tag='NNG', start=5, len=2),
 Token(form='사진', tag='NNG', start=9, len=2),
 Token(form='몇', tag='MM', start=11, len=1),
 Token(form='장', tag='NNB', start=12, len=1),
 Token(form='찍', tag='VV', start=13, len=1),
 Token(form='자고', tag='EC', start=14, len=2),
 Token(form='파리', tag='NNG', start=16, len=2),
 Token(form='가', tag='VV', start=20, len=1),
 Token(form='거', tag='NNB', start=21, len=1),
 Token(form='냐', tag='EC', start=22, len=1),
 Token(form='축구', tag='NNG', start=23, len=2),
 Token(form='협회', tag='NNG', start=25, len=2),
 Token(form='회장', tag='NNG', start=27, len=2),
 Token(form='정치인', tag='NNG', start=33, len=3)]

In [17]:
# 사용자 사전에 추가할 단어를 리스트로 생성
words = ['본인', '축구협회']

In [18]:
# 반복문으로 사용자 사전에 단어를 추가
# [참고] score 매개변수에 전달하는 인수의 값이 클수록 하나의 형태소로 처리(기본값: 0)
for word in words:
    kiwi.add_user_word(word = word, tag = 'NNP', score = 15)

In [19]:
# sens의 첫 번째 원소로 형태소 분석 결과를 tokens에 할당
# [참고] 사용자 사전에 추가한 단어 반영 여부 확인
tokens = kiwi.tokenize(text = sens[i], stopwords = kiwistop)

# tokens 확인
tokens

[Token(form='고작', tag='MAG', start=0, len=2),
 Token(form='본인', tag='NNP', start=2, len=2),
 Token(form='책', tag='NNG', start=4, len=1),
 Token(form='홍보', tag='NNG', start=5, len=2),
 Token(form='사진', tag='NNG', start=9, len=2),
 Token(form='몇', tag='MM', start=11, len=1),
 Token(form='장', tag='NNB', start=12, len=1),
 Token(form='찍', tag='VV', start=13, len=1),
 Token(form='자고', tag='EC', start=14, len=2),
 Token(form='파리', tag='NNG', start=16, len=2),
 Token(form='가', tag='VV', start=20, len=1),
 Token(form='거', tag='NNB', start=21, len=1),
 Token(form='냐', tag='EF', start=22, len=1),
 Token(form='축구협회', tag='NNP', start=23, len=4),
 Token(form='회장', tag='NNG', start=27, len=2),
 Token(form='정치인', tag='NNG', start=33, len=3)]

In [20]:
# tokens의 첫 번째 원소에서 형태소의 형태와 품사 확인
print(f'형태: {tokens[0].form}')
print(f'품사: {tokens[0].tag}')

형태: 고작
품사: MAG


In [21]:
# 텍스트 분석에 사용할 품사(용언과 체언) 목록을 리스트로 생성
pos1, pos2 = ['VV', 'VA'], ['NNG', 'NNP']

In [22]:
# 형태소의 품사가 용언과 체언인 형태소만 선택하고 품사가 용언인 형태소에 
# 종결어미 '다'를 결합하여 리스트로 반환
tokens = [token.form + '다' if token.tag in pos1 else token.form 
          for token in tokens if token.tag in pos1 + pos2]

# tokens 확인
tokens

['본인', '책', '홍보', '사진', '찍다', '파리', '가다', '축구협회', '회장', '정치인']

### 말뭉치 생성

In [23]:
# 말뭉치를 저장할 빈 리스트 생성
corpus = list()

# sens의 원소(문서)를 형태소 분석하고 일부 품사를 선택하여 말뭉치에 추가
for sen in sens:
    tokens = kiwi.tokenize(text = sen, stopwords = kiwistop)
    tokens = [token.form + '다' if token.tag in pos1 else token.form 
              for token in tokens if token.tag in pos1 + pos2]
    corpus.append(tokens)

In [24]:
# 형태소 분석 결과(corpus)의 처음 5개 원소 확인
# [참고] 원본 문서와 결과를 비교하여 사용자 사전에 추가하는 것을 반복해야 함
for i in range(5):
    print(corpus[i])

['본인', '책', '홍보', '사진', '찍다', '파리', '가다', '축구협회', '회장', '정치인']
['회장', '만나다', '정몽규', '우러러보다', '월드컵', '나가다', '전전긍긍', '알다', '택', '소리', '축구협회', '자체', '썩다', '빠지다', '국민', '모두', '분노', '넘치다', '이러다', '피파', '징계', '먹다', '말다', '이번', '월드컵', '포기', '지금', '협회', '조지다', '만들다', '그러다', '올해', '국회', '국정', '감사', '눈', '불', '켜다', '기다리다', '축구협회', '배드민턴', '협회', '박살나다', '저러다', '문체부', '포함', '야당', '여당', '국회의원', '조지다', '조지다', '어설프다', '질문', '협회', '질타', '이상', '질타', '순간', '순간', '뭇매', '맞다']
['크다', '린스', '위약금', '본인', '충당']
['몽규', '비행기', '호텔', '값', '자비', '가다']
['피파에자리좀없슈']


In [25]:
# 반복문으로 원본 문서의 처음 5행을 출력
for i in range(5):
    print(newsReply['contents'].iloc[i], '\n')

고작 본인 책 홍보하고 사진 몇장 찍자고 파리까지 간거냐??? 축협회장이 아니라 정치인같다.ㅉㅉㅉ 

FIFA 회장 만난다고 우리가 정몽규 지를 무슨 우러러 보고
월드컵 못나갈까봐 전전긍긍할줄아나 택도없는소리다
이미 축구협회 자체가 썩어빠졌고 국민들 모두가 분노가
넘쳐서 이럴꺼면 피파에서 징계먹든말든 이번 월드컵 포기하고 지금의 협회부터 조지고 새로만들자고 그러고있는데
다들 지금 올해 국회 국정감사 눈에 불을켜고 기다리고있다
축구협회뿐만 아니라 배드민턴협회도 박살나서 저러고있고
문체부 포함 야당 여당 국회의원 당신들 조질꺼면 제대로 조져라 되도않는 어설픈 질문들고와서 협회들한테 질타같지않은 이상한 질타하는순간 순간 되려 뭇매맞는다 

클린스만 위약금 본인이 꼭 충당해라 

몽규 비행기 호텔값은 자비로 간거니? 아님? 

헤헤~~굽신굽신. 피파에 자리 좀 없슈? 



### 문서-단어 행렬 생성

In [26]:
# corpus의 각 원소(리스트)를 하나의 문자열로 결합하여 corpus에 재할당
# [참고] 문서-단어 행렬 생성하려면 corpus 원소가 문자열이어야 함
corpus = [' '.join(i) for i in corpus]

# corpus의 처음 5개 확인
corpus[0:5]

['본인 책 홍보 사진 찍다 파리 가다 축구협회 회장 정치인',
 '회장 만나다 정몽규 우러러보다 월드컵 나가다 전전긍긍 알다 택 소리 축구협회 자체 썩다 빠지다 국민 모두 분노 넘치다 이러다 피파 징계 먹다 말다 이번 월드컵 포기 지금 협회 조지다 만들다 그러다 올해 국회 국정 감사 눈 불 켜다 기다리다 축구협회 배드민턴 협회 박살나다 저러다 문체부 포함 야당 여당 국회의원 조지다 조지다 어설프다 질문 협회 질타 이상 질타 순간 순간 뭇매 맞다',
 '크다 린스 위약금 본인 충당',
 '몽규 비행기 호텔 값 자비 가다',
 '피파에자리좀없슈']

In [27]:
# 문서별 단어의 tf를 계산하는 객체 생성
# [참고] TfidfVectorizer()는 단어의 tf-idf를 반환
cv = CountVectorizer()

In [28]:
# 단어의 tf를 성분으로 갖는 문서-단어 행렬 생성
dtm = cv.fit_transform(raw_documents = corpus).toarray()

# dtm 확인
# [참고] dtm은 2차원 행렬임
dtm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
# cv 객체에 저장된 단어 목록 확인
# [참고] 알파벳이 모두 소문자로 바뀌었음
cv.get_feature_names_out()

array(['2류', '2부', '3관왕', ..., '힘들이다', '힘주다', '힘줄'], dtype=object)

In [30]:
# dtm을 데이터프레임으로 변환
dtm = pd.DataFrame(data = dtm, columns = cv.get_feature_names_out())

In [31]:
# dtm의 열이름에서 알파벳을 대문자로 변경
dtm.columns = dtm.columns.str.upper()

In [32]:
# dtm의 처음 5행 확인
dtm.head()

,2류,2부,3관왕,3루타,50퍼,X가리에뭐,가격,가관,가난,가누다,...,희생,희한하다,히딩크,히룡,히트,히틀러,힘들다,힘들이다,힘주다,힘줄
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# dtm의 행 개수와 열 개수 확인
dtm.shape

(4032, 4792)

In [34]:
# dtm의 단어별 tf 합계를 내림차순 정렬하고 tfs에 할당
tfs = dtm.sum().sort_values(ascending = False)

In [35]:
# tfs의 상위 10개 및 하위 10개 확인
display(tfs.head(n = 10))
display(tfs.tail(n = 10))

축구      663
축구협회    601
정몽규     501
회장      448
감독      369
홍명보     288
국민      253
가다      243
사퇴      218
협회      217
dtype: int64

사악           1
사약           1
사업이나똑봐로해라    1
사업체          1
사욕           1
사우디          1
사이사이         1
사이언스         1
사이좋다         1
힘줄           1
dtype: int64

In [36]:
# tfs가 20 이하인 인덱스(단어)를 dtm에서 제거
# [참고] 단어 빈도수를 높일수록 dtm의 열(차원) 개수가 감소함
threshold = 20
dtm = dtm.drop(columns = tfs.loc[tfs.le(threshold)].index)

In [37]:
# dtm의 행 개수와 열 개수 확인
# [참고] 단어 빈도수가 매우 작은 일부 단어를 삭제함으로써 차원(열) 축소 가능
# [참고] dtm의 차원을 축소하면 행렬곱 연산을 빠르게 실행할 수 있음
dtm.shape

(4032, 224)

In [38]:
# 현재 작업 경로 확인
os.getcwd()

'/Users/seonghona/Documents/Lectures/Customer/DBR/data'

In [39]:
# corpus, tfs 및 dtm을 하나의 pkl 파일로 저장
pd.to_pickle(obj = [corpus, tfs, dtm], filepath_or_buffer = 'Text_Prep.pkl')

In [40]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['Text_Prep.pkl', 'Naver_News_Reply.pkl', 'Naver_News_List.pkl']

## End of Document